# 🔥 Calories Burn Predictor with Login, Activity Type & Charts

This notebook simulates a calorie prediction system with user login, activity-based prediction using MET values, history tracking via SQLite, and data visualization.

## Step 1: Import Libraries and MET Table

In [ ]:
import sqlite3
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from getpass import getpass

# MET values for different activities
activity_met = {
    "Walking": 3.5,
    "Running": 7.5,
    "Cycling": 6.0,
    "Yoga": 2.5,
    "Gym Workout": 5.5,
    "Swimming": 6.5,
    "Dancing": 4.5,
    "Other": 3.0
}


## Step 2: Set up SQLite Tables for Users and Data

In [ ]:
conn = sqlite3.connect("calories.db")
cursor = conn.cursor()

# User table
cursor.execute('''CREATE TABLE IF NOT EXISTS users (
    username TEXT PRIMARY KEY,
    password TEXT
)''')

# Data table
cursor.execute('''CREATE TABLE IF NOT EXISTS calories_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT,
    age INTEGER,
    gender TEXT,
    height REAL,
    weight REAL,
    duration REAL,
    activity TEXT,
    calories REAL,
    date TEXT
)''')
conn.commit()


## Step 3: User Registration / Login

In [ ]:
# Simulate login/registration
def user_login():
    username = input("Enter username: ")
    password = getpass("Enter password: ")
    cursor.execute("SELECT * FROM users WHERE username=? AND password=?", (username, password))
    if cursor.fetchone():
        print(f"✅ Welcome back, {username}!")
    else:
        print("🔐 User not found. Registering...")
        cursor.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, password))
        conn.commit()
        print(f"✅ Registered new user: {username}")
    return username

username = user_login()


## Step 4: Calorie Calculation Function

In [ ]:
def calculate_calories(met, weight, duration_min):
    duration_hr = duration_min / 60
    return met * weight * duration_hr * 1.05


## Step 5: Enter User Info and Predict

In [ ]:
# Sample input
age = int(input("Enter your age: "))
gender = input("Enter gender (Male/Female): ")
height = float(input("Enter height (cm): "))
weight = float(input("Enter weight (kg): "))
activity = input(f"Select activity {list(activity_met.keys())}: ")
duration = float(input("Duration (in minutes): "))

met = activity_met.get(activity, 3.0)
calories_burned = calculate_calories(met, weight, duration)
print(f"🔥 Calories Burned: {calories_burned:.2f} kcal")

cursor.execute("INSERT INTO calories_data (username, age, gender, height, weight, duration, activity, calories, date) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)",
               (username, age, gender, height, weight, duration, activity, calories_burned, str(datetime.date.today())))
conn.commit()


## Step 6: View Your History

In [ ]:
df = pd.read_sql_query("SELECT * FROM calories_data WHERE username = ?", conn, params=(username,))
df['date'] = pd.to_datetime(df['date'])
df.sort_values("date", inplace=True)
df.tail()


## Step 7: Visualize Your Calories History

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(data=df, x="date", y="calories", hue="activity")
plt.title("Calories Burned Over Time")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,5))
sns.lineplot(data=df, x="date", y="calories", marker="o")
plt.title("Calories Burned Trend")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## ✅ Done!